In [2]:
final_output_path = "D:\\Projects\\SDC\\Term3\\Capstone-Project-SDC-Term3-P3-Udacity\\sim-recs\\final_train"
generator_train_path = "D:\\Projects\\SDC\\Term3\\Capstone-Project-SDC-Term3-P3-Udacity\\sim-recs\\generator_train"
generator_validation_path = "D:\\Projects\\SDC\\Term3\\Capstone-Project-SDC-Term3-P3-Udacity\\sim-recs\\generator_validation"
n_classes = 4

In [3]:
# filter warnings
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

# keras imports
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.mobilenet import MobileNet, preprocess_input
# from keras.applications.mobilenetv2 import MobileNetV2, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import optimizers

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time

Using TensorFlow backend.


In [19]:
# load the user configs
with open('config.json') as f:    
  config = json.load(f)

# config variables
model_name    = config["model"]
weights     = config["weights"]
include_top   = config["include_top"]
train_path    = config["train_path"]
features_path   = config["features_path"]
labels_path   = config["labels_path"]
test_size     = config["test_size"]
results     = config["results"]
model_path    = config["model_path"]
num_classes   = config["num_classes"]
classifier_path = config["classifier_path"]


In [28]:
from keras import backend as K
K.get_session().close()

In [29]:
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

In [30]:
# start time
print ("[STATUS] start time - {}".format(datetime.datetime.now().strftime("%Y-%m-%d %H:%M")))
start = time.time()
print(f"Model name: {model_name}")

# create the pretrained models
# check for pretrained weight usage or not
# check for top layers to be included or not
if model_name == "vgg16":
    base_model = VGG16(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "vgg19":
    base_model = VGG19(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('fc1').output)
    image_size = (224, 224)
elif model_name == "resnet50":
    base_model = ResNet50(weights=weights, include_top=include_top)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    #   model = Model(input=base_model.input, output=base_model.get_layer('flatten').output)
    image_size = (224, 224)
elif model_name == "inceptionv3":
    base_model = InceptionV3(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
    model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
    image_size = (299, 299)
elif model_name == "inceptionresnetv2":
    base_model = InceptionResNetV2(include_top=include_top, weights=weights, input_tensor=Input(shape=(299,299,3)))
    model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
    image_size = (299, 299)
elif model_name == "mobilenet":
    base_model = MobileNet(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3))
    #   model = Model(input=base_model.input, output=base_model.get_layer('custom').output)
    image_size = (224, 224)
    x = base_model.output
    x = Flatten()(x)
elif model_name == "mobilenetv2":
    base_model = MobileNetV2(include_top=include_top, weights=weights, input_tensor=Input(shape=(224,224,3)), input_shape=(224,224,3))  
    #   model.add(Dense(num_class, activation='softmax'))  
    image_size = (224, 224)
elif model_name == "xception":
    base_model = Xception(weights=weights)
    model = Model(input=base_model.input, output=base_model.get_layer('avg_pool').output)
    image_size = (299, 299)
else:
    base_model = None

# Freeze the layers which you don't want to train.
# for layer in base_model.layers[:5]:
for layer in base_model.layers:    
    layer.trainable = False

x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)

predictions = Dense(num_classes, activation="softmax")(x)

# creating the final model 
model_final = Model(inputs = base_model.input, outputs = predictions)
model_final.summary()

# compile the model 
model_final.compile(
    loss = "categorical_crossentropy", 
    #optimizer = optimizers.SGD(lr=0.0001, momentum=0.9, nesterov=True), 
    optimizer = "adam", 
    metrics=["accuracy"])

print ("[INFO] successfully loaded base model and model...")

[STATUS] start time - 2018-09-01 15:30
Model name: resnet50
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
_________________________________________________

In [36]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
#     rescale = 1./255,
    horizontal_flip = False,
    fill_mode = "nearest",
    zoom_range = 0,
    width_shift_range=0.05,
    height_shift_range=0.0,
    rotation_range=2,
    validation_split=0.2)

img_width, img_height = 224, 224
batch_size = 1
train_generator = datagen.flow_from_directory(
    final_output_path, 
    target_size = (img_height, img_width),
    batch_size = batch_size,
    #save_to_dir=generator_train_path,
    subset='training')
print(train_generator.class_indices)
validation_generator  = datagen.flow_from_directory(
    final_output_path, 
    target_size = (img_height, img_width),
    batch_size = batch_size,
    #save_to_dir=generator_validation_path,
    subset='validation')
print(validation_generator.class_indices)

Found 2067 images belonging to 4 classes.
{'green': 0, 'no': 1, 'orange': 2, 'red': 3}
Found 515 images belonging to 4 classes.
{'green': 0, 'no': 1, 'orange': 2, 'red': 3}


In [37]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

# Save the model according to the conditions  
checkpoint = ModelCheckpoint(model_name + "_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [38]:
import tensorflow as tf
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [39]:
if not os.path.exists(generator_train_path):
    os.makedirs(generator_train_path)
else:
    output_file_list = glob.glob(os.path.join(generator_train_path, "*.*"))
    for f in output_file_list:
        os.remove(f)

if not os.path.exists(generator_validation_path):
    os.makedirs(generator_validation_path)
else:
    output_file_list = glob.glob(os.path.join(generator_validation_path, "*.*"))
    for f in output_file_list:
        os.remove(f)

In [40]:
# Train the model
epochs = 50
hist = model_final.fit_generator(
    generator=train_generator,
    epochs = epochs,
    verbose = 1,
    validation_data = validation_generator,
    callbacks = [checkpoint, early])

Epoch 1/50
2067/2067 [==============================] - 97s 47ms/step - loss: 5.9881 - acc: 0.3372 - val_loss: 10.5159 - val_acc: 0.3476

Epoch 00001: val_acc improved from -inf to 0.34757, saving model to resnet50_1.h5
Epoch 2/50
2067/2067 [==============================] - 98s 47ms/step - loss: 5.8522 - acc: 0.3358 - val_loss: 10.5159 - val_acc: 0.3476

Epoch 00002: val_acc did not improve from 0.34757
Epoch 3/50
 998/2067 [=============>................] - ETA: 42s - loss: 5.7619 - acc: 0.3517

KeyboardInterrupt: 